In [1]:
import os
import warnings
warnings.filterwarnings("ignore") 

import pandas as pd
import shutil
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt

from PIL import Image
from sklearn.model_selection import train_test_split

## Подготовка лейблов для обучения модели в формат требуемый YOLOv8x

Координаты парсятся из xml файлов и переводятся в формат:
##### cls cx cy w h

cls - метка класса;

cx - относительная центральная координата бокса по оси x;

cy - относительная центральная координата бокса по оси y;

w - относительная ширина бокса;

h - относительная высота бокса.

In [2]:
df = pd.DataFrame(columns=["image", "cls", "cx", "cy", "w", "h"])

xml_folder = "dataset/dataset/razm"

for filename in os.listdir(xml_folder):
    if filename.endswith(".xml"):
        tree = ET.parse(os.path.join(xml_folder, filename))
        root = tree.getroot()
        
        img_name = root.find("filename").text
        
        for obj in root.findall("object"):
            cls_name = obj.find("name").text
            if cls_name == 'man_with_weapon':
                class_name = '0'
            elif cls_name == 'short_weapons':
                class_name = '1'
            elif cls_name == 'long_weapons':
                class_name = '2'
            else:
                continue
            xmin = int(obj.find("bndbox/xmin").text)
            ymin = int(obj.find("bndbox/ymin").text)
            xmax = int(obj.find("bndbox/xmax").text)
            ymax = int(obj.find("bndbox/ymax").text)
            
            cx = (xmin + xmax) / 2
            cy = (ymin + ymax) / 2
            w = xmax - xmin
            h = ymax - ymin
            
            df = df.append({"image": img_name, "cls": class_name, "cx": cx, "cy": cy, "w": w, "h": h}, ignore_index=True)
            df = df.sort_values(by="image")
print(df.shape)
df.head(100)

(86, 6)


,image,cls,cx,cy,w,h
0,1.jpg,0,680.5,741.5,905,1067
1,1.jpg,2,894.5,809.5,1101,729
2,10.jpeg,2,131.0,43.0,64,28
3,10.jpeg,0,106.5,112.5,93,207
4,11.jpg,0,434.0,280.0,322,382
...,...,...,...,...,...,...
80,7.jpg,2,608.0,289.5,430,269
81,8.jpg,2,277.0,337.5,106,295
82,8.jpg,0,234.5,372.5,191,621
83,9.jpg,0,467.5,365.0,349,492


In [3]:
txt_folder = "dataset/dataset/razm_txt"
os.makedirs(txt_folder, exist_ok=True)

grouped = df.groupby("image")

for img_name, group in grouped:
    txt_file_path = os.path.join(txt_folder, os.path.splitext(img_name)[0] + ".txt")
    with Image.open('dataset/dataset/razm/' + img_name) as img:
        img_width, img_height = img.size
    with open(txt_file_path, "w") as txt_file:
        for index, row in group.iterrows():
            line = f"{row['cls']} {round(row['cx'] / img_width, 6)} {round(row['cy'] / img_height, 6)} {round(row['w'] / img_width, 6)} {round(row['h'] / img_height, 6)}\n"
            txt_file.write(line)

In [ ]:
#Split data for train and val dataset
def move_files(source_folder, destination_folder, files):
    os.makedirs(os.path.join(destination_folder, "images"), exist_ok=True)
    os.makedirs(os.path.join(destination_folder, "labels"), exist_ok=True)
    for file in files:
        source_image_path = os.path.join(source_folder, 'images', file)
        source_label_path = os.path.join(source_folder, "labels", file.replace(".jpg", ".txt").replace(".jpeg", ".txt").replace(".png", ".txt"))
        destination_image_path = os.path.join(destination_folder, "images", file)
        destination_label_path = os.path.join(destination_folder, "labels", file.replace(".jpg", ".txt").replace(".jpeg", ".txt").replace(".png", ".txt"))
        shutil.move(source_image_path, destination_image_path)
        shutil.move(source_label_path, destination_label_path)

source_dir = '/home/jupyter/work/resources/train'
val_dir = "/home/jupyter/work/resources/val"

image_files = [f for f in os.listdir(source_dir + '/images')]

image_files_train, image_files_val = train_test_split(image_files, test_size=0.15, random_state=42)

move_files(source_dir, val_dir, image_files_val)